In [101]:
%pip uninstall hopsworks hsfs hsml -y

%pip install hopsworks

Found existing installation: hopsworks 4.1.4
Uninstalling hopsworks-4.1.4:
  Successfully uninstalled hopsworks-4.1.4
Note: you may need to restart the kernel to use updated packages.
  Using cached hopsworks-4.1.4-py3-none-any.whl.metadata (11 kB)
Using cached hopsworks-4.1.4-py3-none-any.whl (640 kB)
Note: you may need to restart the kernel to use updated packages.


In [102]:
import dotenv

dotenv.load_dotenv()

True

In [103]:
from datetime import datetime, timedelta, timezone
import pandas as pd
from xgboost import XGBRegressor
import hopsworks
import json
import os

In [104]:
import hopsworks

project = hopsworks.login(project="dbillLab1")
fs = project.get_feature_store()

2025-01-08 03:41:50,403 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-08 03:41:50,409 INFO: Initializing external client
2025-01-08 03:41:50,409 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-08 03:41:51,816 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169559


In [105]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("alanvourch/tmdb-movies-daily-updates")

# List files in the downloaded dataset directory
files = os.listdir(path)
print("Files in dataset directory:", files)

# Read the first file
updated_df = pd.read_csv(os.path.join(path, files[0]))

# Print the first 5 rows
updated_df.head()

Files in dataset directory: ['TMDB_all_movies.csv']


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,budget,imdb_id,...,spoken_languages,cast,director,director_of_photography,writers,producers,music_composer,imdb_rating,imdb_votes,poster_path
0,2,Ariel,7.1,335.0,Released,1988-10-21,0.0,73.0,0.0,tt0094675,...,suomi,"Heikki Salomaa, Hanna Jokinen, Matti Pellonpää...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Aki Kaurismäki,NaN,7.4,8870.0,/ojDg0PGvs6R9xYFodRct2kdI6wC.jpg
1,3,Shadows in Paradise,7.3,400.0,Released,1986-10-17,0.0,74.0,0.0,tt0092149,...,"English, suomi, svenska","Teuvo Rissanen, Malla Hukkanen, Olli Varja, Ju...",Aki Kaurismäki,Timo Salminen,Aki Kaurismäki,Mika Kaurismäki,NaN,7.5,7654.0,/nj01hspawPof0mJmlgfjuLyJuRN.jpg
2,5,Four Rooms,5.9,2650.0,Released,1995-12-09,4257354.0,98.0,4000000.0,tt0113101,...,English,"Lili Taylor, Valeria Golino, Marisa Tomei, Mar...","Quentin Tarantino, Robert Rodriguez, Allison A...","Andrzej Sekula, Rodrigo García, Guillermo Nava...","Quentin Tarantino, Robert Rodriguez, Allison A...","Quentin Tarantino, Lawrence Bender, Alexandre ...",Combustible Edison,6.7,113089.0,/pyCk5JgtRZwRxnXwfrvyzukaKue.jpg
3,6,Judgment Night,6.5,333.0,Released,1993-10-15,12136938.0,109.0,21000000.0,tt0107286,...,English,"Stephen Dorff, Everlast, Will Zahrn, Emilio Es...",Stephen Hopkins,Peter Levy,"Jere Cunningham, Lewis Colick","Gene Levy, Marilyn Vance, Lloyd Segan",Alan Silvestri,6.6,19456.0,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg
4,8,Life in Loops (A Megacities RMX),7.5,27.0,Released,2006-01-01,0.0,80.0,42000.0,tt0825671,...,"English, हिन्दी, 日本語, Pусский, Español",NaN,Timo Novotny,Wolfgang Thaler,"Michael Glawogger, Timo Novotny","Ulrich Gehmacher, Timo Novotny",NaN,8.2,284.0,/7ln81BRnPR2wqxuITZxEciCe1lc.jpg


In [106]:
updated_df['status'].value_counts()

status
Released           1021903
Planned               5275
Post Production       5179
In Production         5060
Rumored               1093
Canceled               600
Name: count, dtype: int64

In [107]:
df_new = updated_df[updated_df['status'] == 'Post Production'].copy()
df_new['release_date'] = pd.to_datetime(df_new['release_date'], errors='coerce')
df_new.dropna(subset=['release_date'], inplace=True)

In [108]:
# Changing the release_date to release_year
df_new['release_year'] = pd.to_datetime(df_new['release_date']).dt.year

for col in df_new.columns:
    if col in df_new.columns:
        df_new[col] = df_new[col].where(pd.notna(df_new[col]), None)

df_new['timestamp'] = datetime.now()

In [109]:
tmdb_movie_fg = fs.get_or_create_feature_group(
    name='tmdb_upcoming_movies',
    description='Movie information including director, actors, genres, etc.',
    version=4,
    primary_key=['id'],
    online_enabled=True,
)

tmdb_movie_fg.save(df_new)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1169559/fs/1160262/fg/1394752








Uploading Dataframe: 100.00% |██████████| Rows 760/760 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: tmdb_upcoming_movies_4_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1169559/jobs/named/tmdb_upcoming_movies_4_offline_fg_materialization/executions


(Job('tmdb_upcoming_movies_4_offline_fg_materialization', 'SPARK'), None)

%6|1736318550.493|FAIL|rdkafka#producer-7| [thrd:ssl://51.161.80.189:9093/bootstrap]: ssl://51.161.80.189:9093/0: Disconnected (after 100714ms in state UP, 1 identical error(s) suppressed)
